# Final Project

In [21]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [22]:
#Importing the original file

fname = "business.csv"
business_df = pd.read_csv(fname)

In [23]:
#Removing unneccesary columns and renaming to layman terms, getting all food/drinks related commercial places

business_df = business_df[['doing_bus_as_name','zip','naics_description','lat','lon']]
business_df = business_df.loc[(business_df['naics_description'] == 'full-service restaurants') |
               (business_df['naics_description'] == 'cafeterias') | 
               (business_df['naics_description'] == 'food services & drinking places') |
               (business_df['naics_description'] == 'limited-service eating places') |
               (business_df['naics_description'] == 'limited-service restaurants')  |
               (business_df['naics_description'] == 'mobile food services') |
               (business_df['naics_description'] == 'drinking places (alcoholic beverages)') |
               (business_df['naics_description'] == 'snack & nonalcoholic beverage bars')]
business_df.rename(columns = {'doing_bus_as_name':'Business title','naics_description':'Type of Place'}, inplace=True)
business_df = business_df.reset_index(drop=True)

In [24]:
#Adding the 5 columns for the different time brackets

business_df.insert(2,'AM early','Null')
business_df.insert(3,'AM peak','Null')
business_df.insert(4,'Mid-day','Null')
business_df.insert(5,'PM peak','Null')
business_df.insert(6,'PM late','Null')

In [25]:
#generalising the different categories into 3 categories: Only Food, Food & Drinks, Only Drinks
business_df.loc[(business_df['Type of Place'] == 'mobile food services') |
                (business_df['Type of Place'] == 'cafeterias') |
                (business_df['Type of Place'] == 'snack & nonalcoholic beverage bars') |
                (business_df['Type of Place'] == 'limited-service eating places') |
                (business_df['Type of Place'] == 'limited-service restaurants') 
                , 'Type of Place'] = 'Only Food'

business_df.loc[(business_df['Type of Place'] == 'full-service restaurants') 
                , 'Type of Place'] = 'Food & Drinks'

business_df.loc[(business_df['Type of Place'] == 'drinking places (alcoholic beverages)') |
                (business_df['Type of Place'] == 'food services & drinking places') 
                , 'Type of Place'] = 'Only Drinks'


In [31]:
#Assigning values to different time brackets by assuming popularity of the place in that time zone

business_df.loc[(business_df['Type of Place'] == 'Only Food'), 
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('3','25','20','18','5')
business_df.loc[(business_df['Type of Place'] == 'Food & Drinks'),
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('7','28','25','35','25')
business_df.loc[(business_df['Type of Place'] == 'Only Drinks'),
                ('AM early','AM peak','Mid-day','PM peak','PM late')] = ('27','6','3','27','30') 

In [32]:
#business_df.insert(2,'Total Score','Null')
business_df.loc[(business_df['Type of Place'] == 'Only Food'), 
                'Total Score'] = '70'
business_df.loc[(business_df['Type of Place'] == 'Food & Drinks'),
                'Total Score'] = '119'
business_df.loc[(business_df['Type of Place'] == 'Only Drinks'),
                'Total Score'] = '93'

In [33]:
business_df.to_csv('/Users/Devyaanshu/Pr_007/restaurants_data.csv')

In [34]:
business_df

,Business title,zip,AM early,AM peak,Mid-day,PM peak,PM late,Type of Place,lat,lon,Total Score
0,c r e a m,92115-1939,3,25,20,18,5,Only Food,32.767243,-117.096294,70
1,snow cones y raspados,92113-2911,3,25,20,18,5,Only Food,32.697974,-117.096250,70
2,jalapeno taco shop,92104-2047,7,28,25,35,25,Food & Drinks,32.748683,-117.126968,119
3,tacos el campechano inc,91950-1121,3,25,20,18,5,Only Food,32.697974,-117.096250,70
4,up2you cafe llc,92111-5000,3,25,20,18,5,Only Food,32.718370,-117.157817,70
5,awash ethiopian restaurant,92104-1102,7,28,25,35,25,Food & Drinks,32.677079,-117.107167,119
6,gueros taco shop,92102-4019,3,25,20,18,5,Only Food,32.890137,-117.150877,70
7,the fire spot,92111-2315,7,28,25,35,25,Food & Drinks,32.773069,-117.156144,119
8,up2you cafe llc,92111-1545,27,6,3,27,30,Only Drinks,33.016118,-117.075608,93
9,cold beers & cheeseburgers,92101-6910,7,28,25,35,25,Food & Drinks,32.708791,-117.160357,119
